<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/TCN_for_Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the TCN model
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size=3):
        super(TCN, self).__init__()
        self.tcn = nn.Sequential(
            nn.Conv1d(input_size, num_channels, kernel_size=kernel_size, padding=1, dilation=1),
            nn.ReLU(),
            nn.Conv1d(num_channels, num_channels, kernel_size=kernel_size, padding=2, dilation=2),
            nn.ReLU(),
            nn.Conv1d(num_channels, output_size, kernel_size=kernel_size, padding=4, dilation=4)  # Adjust this layer
        )

    def forward(self, x):
        return self.tcn(x)

# Generate synthetic data
input_size = 1
output_size = 1
num_channels = 32
num_samples = 1000
sequence_length = 10

X = torch.randn(num_samples, input_size, sequence_length)
y = torch.randn(num_samples, output_size, sequence_length)

dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the model, loss function, and optimizer
model = TCN(input_size=input_size, output_size=output_size, num_channels=num_channels)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)

        # Ensure output and target sizes match
        if outputs.shape != targets.shape:
            targets = targets[:, :, :outputs.shape[2]]

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training completed!")